## Week 9
### (1) Coupled carbon model and (2) Smoothing data
We will include land and ocean sinks and allow the rate of respiration to change with temperature

Relevant equations from last time

\begin{equation*}
C_b(t) = C_b(t-1) + (NPP - R_h)
\end{equation*}


\begin{equation*}
NPP(t) = NPP(0) [ 1+\beta \ln \frac {CO_2(t)}{CO_2(0)} ]
\end{equation*}



In [ ]:
import numpy as np
import matplotlib.pyplot as py

Read in the historical CO2 data

In [ ]:
filedata = np.loadtxt('ObsCO2.txt', skiprows=1, delimiter='\t')
dateCO2 = filedata[:,0]
CO2obs = filedata[:,1]

In [ ]:
print('start date', dateCO2[0], 'end date', dateCO2[-1])

Read in the historical and projected fossil fuel emissions

In [ ]:
filedata = np.loadtxt('Fossilhistandrcp85.txt', skiprows=1, delimiter='\t')
date = filedata[:,0]
ff = filedata[:,1]
print('start date', date[0], 'end date', date[-1])

In [ ]:
nyears = len(date)
print('number of years', nyears)

#### Initialize the model, this time with both Cb and Ca

Assume that we started at steady state. In 1765, dC/dt = 0 = npp - rh

Cb = npp*tau

In [ ]:
Cb = np.zeros(nyears)
Ca = np.zeros(nyears)

Tp = np.zeros(nyears)
npp = np.zeros(nyears)
rh = np.zeros(nyears)

npp_initial = 50 #units PgC/yr
taub = 20 #units years, initial turnover time for biosphere. Tweak this
beta = 0.6 #sensitivity of NPP to CO2
dt = 1.0 #time increment in years
alpha = 0.0066 #K/ppm sensitivity of T to change in CO2 ppm
ppmtoPgC = 2.1 #conversion, 1 ppm CO2 = 2.1 PgC

#initial conditions
Cb[0] = npp_initial*taub
Ca[0] = 280*ppmtoPgC
Tp[0] = 15.0
npp[0] = npp_initial
rh[0] = npp_initial

Run the model (just the atmospheric carbon and temperature for now)

In [ ]:
for t in range(1, nyears):  #starting at 1766
    Ca[t] = Ca[t-1] + ff[t]*dt
    Tp[t] = Tp[0] + alpha*(Ca[t-1]-Ca[0])/ppmtoPgC
    
    
CO2model = Ca/ppmtoPgC

#print('Land carbon accumulation', -1*np.sum(Fland))

Plot

In [ ]:
fig = py.figure(1)
py.plot(dateCO2, CO2obs, color='black', label='Observed CO2')
py.plot(date, CO2model, color='blue', label='Modeled CO2')
py.xlabel('Year')
py.ylabel('CO2 (ppm)')
py.title('Trajectory of atmospheric CO2')
py.legend()

Our model doesn't include the land sink for carbon, so our observed CO2 is lower than the model

In [ ]:
fig = py.figure(2)
py.plot(date, Tp, color='red', label='Modeled Temperature')
py.xlabel('Year')
py.ylabel('Temperature (deg C)')
py.title('Trajectory of global temperature')

If we didn't have sinks for carbon, warming would be 7 degrees by 2100

### Day 2: See Python script coupledmodelv4.py
includes coupling with ocean and land

line 108: depth of ocean should be 3800

### Day 3: Smoothing data
Create an array with 0 through 50 (A) but with random noise added (B). Smooth B into C

In [ ]:
A = np.arange(50) 
N = np.random.randn(50)

B = A + N
tile3 = np.array([1,1,1]) #having a longer tile is more smoothing
tile3 = tile3/len(tile3)

tile5 = np.array([1,1,1,1,1])
tile5 = tile5/len(tile5)

C3 = np.convolve(B, tile3, mode='valid')
C5 = np.convolve(B, tile5, mode='valid')

In [ ]:
print(len(B))
print(len(C3)) #C is shorter - one element cut off at either end
print(len(C5))

In [ ]:
A_short = A[1:-1]
A_shorter = A[2:-2]

In [ ]:
fig = py.figure(3)
py.plot(A, B, label='original')
py.plot(A_short, C3, label='smoothed')
py.plot(A_shorter, C5, label='more smoothed')
py.legend()

Side note: Using tile [-1, 0, 1] calculates the derivative

Revisit the Mauna Loa dataset

In [ ]:
filename = 'maunaloadata.csv'
data = np.loadtxt(filename, skiprows=59, delimiter = ',')
date = data[:,3] #accessing all rows and column 3
dataCO2 = data[:,8] #accessing all rows and column 8

In [ ]:
width = 61 #5 year smoothing
tile = np.ones(width)
tile = tile/np.sum(tile)

CO2_smoothed = np.convolve(dataCO2, tile, mode='valid')

crop = int((width-1)/2) #range of valid smoothed data
date_cropped = date[crop:-crop] #cut off the unsmoothed ends

In [ ]:
fig = py.figure(4)
py.plot(date, dataCO2, label='original')
py.plot(date_cropped, CO2_smoothed, label='5-yr smoothed')
py.legend()

Take derivative of smoothed line

In [ ]:
dt = 1/12
width = 3
tile_d = np.array([1,0,-1])

CO2_gr = np.convolve(CO2_smoothed, tile_d, mode='valid') #co2 growth rate

crop = int((width-1)/2) #range of valid smoothed data
date_d = date_cropped[crop:-crop] #cut off the unsmoothed ends

In [ ]:
print(len(CO2_smoothed))
print(len(date_d))

In [ ]:
fig = py.figure(5)
py.plot(CO2_gr, label='CO2 growth rate (ppm/month)')
py.legend()

## Week 10
Polynomial fit on CO2 data. A 4th-degree polynomial will remove the seasonal cycle and capture most of the long-term trend. 

In [ ]:
fitcoeff = np.polyfit(date, dataCO2, 4)
firstpolyf = np.poly1d(fitcoeff) #function
firstpolyfit = firstpolyf(date)

In [ ]:
fig = py.figure(6)
py.plot(date, dataCO2, label='raw data')
py.plot(date, firstpolyfit, label='trend')
py.xlabel('Year')
py.ylabel('CO2 (ppm)')
py.title('CO2 trend')
py.legend()

Plot the residual after removing the long-term trend (now we're just seeing the seasonal cycle

In [ ]:
residualCO2 = dataCO2 - firstpolyfit

fig = py.figure(7)
py.plot(date, residualCO2, label='residual')
py.xlabel('Year')
py.ylabel('CO2 (ppm)')
py.title('CO2 residual')
py.legend()

Remove the seasonal cycle: calculate the mean for each month of the year

In [ ]:
#start at January 1959
residualCO2 = residualCO2[10:]
dataCO2 = dataCO2[10:]
date = date[10:]

nmonths = 12
nyears = int(len(residualCO2)/12)


In [ ]:
annualcycle = np.zeros(nmonths)
npoints = np.zeros(nmonths) #count how many of each month

for t in range(nyears):
    for i in range(nmonths):
        annualcycle[i] = annualcycle[i] + residualCO2[t*12 + i]
        npoints[i] = npoints[i] + 1

annualcycle = annualcycle/npoints

In [ ]:
fig = py.figure(8)
py.plot(annualcycle)
py.xlabel('Month')
py.ylabel('CO2 (ppm)')
py.title('CO2 annual cycle')

In [ ]:
repeatcycle = np.tile(annualcycle, nyears)
dataCO2nocycle = dataCO2 - repeatcycle

In [ ]:
fig = py.figure(9)
py.plot(date, dataCO2nocycle)
py.xlabel('Month')
py.ylabel('CO2 (ppm)')
py.title('CO2 annual cycle')

Calculate derivative